In [1]:
pip install -q wikipedia

Note: you may need to restart the kernel to use updated packages.


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [3]:
api_wrapper = WikipediaAPIWrapper(top_k_results=1, doc_content_chars_max=500)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)
wiki.invoke({'query': 'llamaindex'})

'Page: Vector database\nSummary: A vector database management system (VDBMS) or simply vector database or vector store is a database that can store vectors (fixed-length lists of numbers) along with other data items. Vector databases typically implement one or more Approximate Nearest Neighbor (ANN) algorithms, so that one can search the database with a query vector to retrieve the closest matching database records.\nVectors are mathematical representations of data in a high-dimensional space. In t'

# Reasoning and Acting (React)

## Creating a ReAct Agent

In [4]:
#pip install langchainhub -q

In [5]:
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [6]:
from langchain.prompts import PromptTemplate
from langchain import hub
from langchain.agents import Tool, AgentExecutor, initialize_agent, create_react_agent
from langchain.tools import DuckDuckGoSearchRun, WikipediaQueryRun
from langchain.utilities import WikipediaAPIWrapper
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name='gpt-3.5-turbo', temperature=0)

template = '''
Answer the following questions in GERMAN as best uou can.
Questions: {q}
'''
prompt_template = PromptTemplate.from_template(template)
prompt = hub.pull('hwchase17/react')
# print(type(prompt))
# print(prompt.input_variables)
# print(prompt.template)

# 1. Python REPL Tool (for executing Python code)
python_repl = PythonREPLTool()
python_repl_tool = Tool(
    name = 'Python REPL',
    func = python_repl.run,
    description = 'Useful when you need to use Python to answer a question. You Should input Python code.'
)

# 2. Wikipedia Tool (for searching wikipedia)
api_wrapper = WikipediaAPIWrapper()
wikipedia = WikipediaQueryRun(api_wrapper=api_wrapper)
wikipedia_tool = Tool(
    name='Wikipedia',
    func=wikipedia.run,
    description='Useful for when you need to look up a topic, country, or person on wikipedia'
)
# 3. DuckDuckGo Search Tool (for general web searches)
search = DuckDuckGoSearchRun()
duckduckgo_tool = Tool(
    name='DuckDuckGo Search',
    func=search.run,
    description='Useful for when you need to perform an internet search to find information that another tool cant provide'
)

tools = [python_repl_tool, wikipedia_tool, duckduckgo_tool]

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations= 10
)

In [7]:
question = 'Generate the first 20 numbers in the Fibonacci series'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I need to generate the Fibonacci series in German.
Action: Python REPL
Action Input: 
```python
def fibonacci(n):
    if n <= 1:
        return n
    else:
        return fibonacci(n-1) + fibonacci(n-2)

fibonacci_series = [fibonacci(i) for i in range(20)]
fibonacci_series
```I now have the first 20 numbers in the Fibonacci series in German.
Final Answer: Die ersten 20 Zahlen in der Fibonacci-Folge sind 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, 55, 89, 144, 233, 377, 610, 987, 1597, 2584, 4181.

> Finished chain.


In [8]:
question = 'Who is the current prime minister of the UK?'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...
I need to find out who the current prime minister of the UK is in German.
Action: DuckDuckGo Search
Action Input: "Aktueller Premierminister Großbritannien"In Umfragen liegt Premierminister Sunak deutlich hinter der Oppositionspartei Labour um Starmer. 09.01.2024 | 2:16 min 2024 beginnt mit einem Mega-Streik in Großbritannien : Die Assistenz-Ärzte ... Die Premierministerin von Großbritannien und Nordirland ist Theresa May. Sie ist seit Juli 2016 Premierministerin und steht dem britischen Kabinett vor. ... Der britische Premierminister ist der amtierende Regierungschef des Vereinigten Königreichs. Seit der Gründung des Amtes im Jahr 1721 durch Robert Walpole hat es 13 Amtsinhaber gegeben ... Premierminister Rishi Sunak warnt vor dem Erstarken extremistischer Kräfte. Bei einer Parlamentsnachwahl hatte zuletzt ein Kandidat mit antiisraelischer Haltung gewonnen. September 2022, nach der Sommerpause des Parlaments, soll der neue Tory-Vorsitzende und d

D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


Stand: 13.11.2023 13:57 Uhr. Der frühere britische Premier David Cameron wird als neuer Außenminister wieder Teil der Regierung. Er übernimmt das Amt von James Cleverly, der die umstrittene ... Eine Kabinettsumbildung in Großbritannien kommt einem politischen Erdbeben gleich: Regierungschef Rishi Sunak hat Innenministerin Suella Braverman entlassen - und macht Ex-Premierminister David ... Schätzung für 2022 Zuwanderung nach Großbritannien auf Rekordniveau Stand: 25.05.2023 14:33 Uhr Dennoch ist die Nettozuwanderung 2022 auf ein Rekordniveau gestiegen. Knalleffekt in Großbritannien: Der frühere Premierminister David Cameron wird neuer britischer Außenminister. Das gab die Regierung in London am Montag im Zuge einer Kabinettsumbildung bekannt. Der britische Premierminister Rishi Sunak bildet sein Kabinett um - und entlässt nicht nur Innenministerin Suella Braverman, sondern holt auch seinen Vorgänger mit in die Regierung. Braverman ...The search results are still not providing a clear an

D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


Januar 2020 trat Großbritannien aus der EU aus. ... 100 Tage ist Rishi Sunak britischer Premierminister und im Land türmen sich die Probleme. ... 27.12.2023 • 05:01 Uhr. Großbritannien Das ... Heute schauen wir uns mal an, wer in England regiert. Es gibt da so einige interessante Fakten zu entdecken, also lass uns direkt loslegen! Die Premierministerin von Großbritannien und Nordirland ist Theresa May. Sie ist seit Juli 2016 Premierministerin und steht dem britischen Kabinett vor. Sie ist zugleich auch Leader of the Conservative ... Migration in Großbritannien auf Rekordniveau. 25.05.2023. Vor allem konservative Politiker warben seinerzeit für den Austritt Großbritanniens aus der EU mit dem Argument, die Zahl der ... Was für einen Machtwechsel in London spricht. 2024 wählt das Vereinigte Königreich ein neues Parlament. Die Tories um Premierminister Rishi Sunak haben wenig Grund zu Optimismus. Zu groß ist ... September 2023, 20:50 Uhr. Veröffentlicht am 549 Kommentare. Artikel hören. Ar

...

D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


Was für einen Machtwechsel in London spricht. 2024 wählt das Vereinigte Königreich ein neues Parlament. Die Tories um Premierminister Rishi Sunak haben wenig Grund zu Optimismus. Zu groß ist ... Knalleffekt in Großbritannien: Der frühere Premierminister David Cameron wird neuer britischer Außenminister. Das gab die Regierung in London am Montag im Zuge einer Kabinettsumbildung bekannt. Wahljahr in UK Grossbritannien steht 2024 womöglich vor einem Regierungswechsel. Grossbritannien steht 2024 womöglich vor einem Regierungswechsel. In diesem Jahr stehen in Grossbritannien Wahlen ... Der britische Premierminister baut sein Kabinett radikal um: Suella Braverman muss gehen, James Cleverly wird Innenminister. Neuer Außenminister wird ein früherer Regierungschef. Großbritannien "Der wahrscheinliche nächste Premierminister" Großbritannien Sunaks Tories verlieren bei Nachwahlen gegen Labour; Zur Themenseite Großbritannien. Gutscheine: Booking.comThe search results are not providing a clear answ

D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


Grossbritannien stutzt die eigenen Ambitionen in der Klimapolitik zurück. Bettina Schulz, London 19.02.2024, 11.29 Uhr 4 min. Hören. Was für einen Machtwechsel in London spricht. 2024 wählt das Vereinigte Königreich ein neues Parlament. Die Tories um Premierminister Rishi Sunak haben wenig Grund zu Optimismus. Zu groß ist ... Wahljahr in UK Grossbritannien steht 2024 womöglich vor einem Regierungswechsel. Grossbritannien steht 2024 womöglich vor einem Regierungswechsel. In diesem Jahr stehen in Grossbritannien Wahlen ... Großbritannien "Der wahrscheinliche nächste Premierminister" Großbritannien Sunaks Tories verlieren bei Nachwahlen gegen Labour Zur Themenseite Großbritannien Cameron war von Mai 2010 bis Juli 2016 Premierminister des Vereinigten Königreichs. ... Großbritannien "Der wahrscheinliche nächste Premierminister" Großbritannien Sunaks Tories verlieren bei ...

> Finished chain.


D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\curl_cffi\aio.py:192: UserWarning: Curlm alread closed! quitting from process_data
  warnings.warn("Curlm alread closed! quitting from process_data")


In [9]:
print(output['input'])


Answer the following questions in GERMAN as best uou can.
Questions: Who is the current prime minister of the UK?



In [10]:
output['output']

'Agent stopped due to iteration limit or time limit.'

In [11]:
question = 'Tell me about Napolean bonaparte?'
output = agent_executor.invoke({
   'input': prompt_template.format(q=question) 
})



> Entering new AgentExecutor chain...
I need to find information about Napoleon Bonaparte in German.
Action: Wikipedia
Action Input: Napoleon Bonaparte German WikipediaPage: Napoleon II
Summary: Napoleon II (Napoléon François Joseph Charles Bonaparte; 20 March 1811 – 22 July 1832) was the disputed Emperor of the French for a few weeks in 1815. He was the son of Emperor Napoleon I and Empress Marie Louise, daughter of Emperor Francis I of Austria. Napoleon II had been Prince Imperial of France and King of Rome since birth. After the fall of his father, he lived the rest of his life in Vienna and was known in the Austrian court as Franz, Duke of Reichstadt  for his adult life (from the German version of his second given name, along with a title his grandfather granted him in 1818).  He was posthumously given the nickname L'Aiglon ("the Eaglet") after the popular Edmond Rostand play, L'Aiglon.
When Napoleon I tried to abdicate on 4 April 1814, he said that his son would rule as emperor.

# Vector Databases

In [12]:
# Pinecone is a vector database that is designed for storing a query of high dimensional
# vectors. It provides fast and efficient semantic search over vector embeddings by integrating open ai
# open ai LLMs, we fine deep learning capability for embedding generation with efficient vector search
# storage and retrieval .

# Vector databases are a new type of database, designed to store and query unstructured data.
# Unstructured data is data that does not have a fixed schema, such as text, images and audio.

# Pipeline for Vector Databases
# Vector databases use a combination of different optimized algorithms that all participate in
# Approximate Nearest Neighbor (ANN) search.

# Steps 1> Embedding, 2> Indexing, 3> Querying




In [13]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [14]:
pip install -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install --upgrade -q pinecone-client

Note: you may need to restart the kernel to use updated packages.


In [16]:
pip show pinecone-client

Name: pinecone-client
Version: 3.1.0
Summary: Pinecone client and SDK
Home-page: https://www.pinecone.io
Author: Pinecone Systems, Inc.
Author-email: support@pinecone.io
License: Apache-2.0
Location: D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages
Requires: certifi, tqdm, typing-extensions, urllib3
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [17]:
from pinecone import Pinecone
pc = Pinecone()
#pc = Pinecone(api_key='d5fa5468-8b37-4add-a187-42f0e7c3c675')
pc.list_indexes()

D:\Work\Gre\UTD\Courses\Spring_II\MIS6382\Softwares\Python\Python311\class\Lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


{'indexes': [{'dimension': 1536,
              'host': 'langchain-yqpbghn.svc.gcp-starter.pinecone.io',
              'metric': 'cosine',
              'name': 'langchain',
              'spec': {'pod': {'environment': 'gcp-starter',
                               'pod_type': 'starter',
                               'pods': 1,
                               'replicas': 1,
                               'shards': 1}},
              'status': {'ready': True, 'state': 'Ready'}}]}

In [18]:
pip install -q tqdm

Note: you may need to restart the kernel to use updated packages.


In [19]:
from tqdm.autonotebook import tqdm

In [20]:
pc.list_indexes()[0]

{'dimension': 1536,
 'host': 'langchain-yqpbghn.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'langchain',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [21]:
pc.describe_index('langchain')

{'dimension': 1536,
 'host': 'langchain-yqpbghn.svc.gcp-starter.pinecone.io',
 'metric': 'cosine',
 'name': 'langchain',
 'spec': {'pod': {'environment': 'gcp-starter',
                  'pod_type': 'starter',
                  'pods': 1,
                  'replicas': 1,
                  'shards': 1}},
 'status': {'ready': True, 'state': 'Ready'}}

In [24]:
from pinecone import PodSpec
index_name = 'langchain'

if index_name not in pc.list_indexes().names():
    print(f'Creating index {index_name}')
    pc.create_index(
    name=index_name,
    dimension=1536,
    metric='cosine',
    spec=PodSpec(
        environment='gcp-starter'
    )
    )
    
    print('Index Created!')
else:
    print(f'Index {index_name} already exists!')
    


Index langchain already exists!


In [25]:
index_name = 'lanchain'
if index_name in pc.list_indexes().names():
    print(f'Deleting index {index_name} ...')
    pc.delete_index(index_name)
    print('Done')
else:
    print(f'Index {index_name} does not exist')

Index lanchain does not exist


In [29]:
index = pc.Index(index_name)
index.describe_index_status()

NotFoundException: (404)
Reason: Not Found
HTTP response headers: HTTPHeaderDict({'content-type': 'text/plain; charset=utf-8', 'access-control-allow-origin': '*', 'vary': 'origin,access-control-request-method,access-control-request-headers', 'access-control-expose-headers': '*', 'X-Cloud-Trace-Context': '8dfa9cb750effc3cb73c4fd3b956e524', 'Date': 'Sun, 03 Mar 2024 03:35:12 GMT', 'Server': 'Google Frontend', 'Content-Length': '83', 'Via': '1.1 google', 'Alt-Svc': 'h3=":443"; ma=2592000,h3-29=":443"; ma=2592000'})
HTTP response body: {"error":{"code":"NOT_FOUND","message":"Resource lanchain not found"},"status":404}


# Working with Vectors

In [ ]:
import random
vectors = [[random.random() for _ in range(1536)] for v in range(5)]
